<a href="https://colab.research.google.com/github/Ivan-Nebogatikov/HumanActivityRecognition/blob/master/Activity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

features = pd.read_csv('/final_ecg_data.csv')
features = features[features['ECG'] > 0.]
activities = list(sorted(set(features['activity'])))
print("Activities:", activities)
#features['activity'] = list(map(lambda x: activities.index(x), features['activity']))
#features = features[features['activity'] != 3]
#features = features[features['activity'] != 4]
#print(features.head(5)) 
print('The shape of our features is:', features.shape)

features.describe()

Activities: ['down the stairs', 'inactive', 'run', 'up the stairs', 'walk']
The shape of our features is: (128694, 2)


,ECG
count,128694.000000
mean,0.055353
std,0.038701
min,0.000031
25%,0.031952
50%,0.043823
75%,0.078003
max,0.124969


In [0]:
import numpy as np
labels = np.array(features['activity']) # значения, которые надо будет предсказывать
features= features.drop('activity', axis = 1)
feature_list = list(features.columns)
features = np.array(features)

In [0]:
Разделяем на обучающую (85%) и тестовую (15%) выборки

In [0]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.15, random_state = 242)
print('Training Features Shape:', train_features.shape)
print('Testing Features Shape:', test_features.shape)

Training Features Shape: (109389, 1)
Testing Features Shape: (19305, 1)


Строим случайный лес

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100, random_state = 1) # Классификатор с 100 деревьями
rf.fit(train_features, train_labels);


Вспомогательная функция для построения таблицы с AUC

In [0]:
import pandas as pd
import numpy as np
from scipy import interp

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer

def class_report(y_true, y_pred, y_score=None, average='micro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    print("Accuracy:", accuracy_score(y_true, y_pred))

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = 5
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred'] = pred_cnt
    class_report_df['pred'].iloc[-1] = total

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_it])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    print(class_report_df)

Проверка на тестовых данных. 

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn import metrics

predictions = list(rf.predict(test_features))
pred_prob = rf.predict_proba(test_features)

class_report(
    y_true=test_labels, 
    y_pred=np.asarray(predictions), 
    y_score=pred_prob, average='micro')

Accuracy: 0.4384874384874385
                 precision    recall  f1-score  support     pred       AUC
down the stairs   0.161919  0.043165  0.068160   2502.0    667.0  0.564738
inactive          0.487542  0.725139  0.583064   1619.0   2408.0  0.926043
run               0.599251  0.622252  0.610535   5914.0   6141.0  0.771685
up the stairs     0.243863  0.124219  0.164596   3679.0   1874.0  0.595384
walk              0.370785  0.544804  0.441257   5591.0   8215.0  0.624040
avg / total       0.399309  0.438487  0.403929  19305.0  19305.0  0.751006


Хуже всего распознаются движения вверх/вниз по лестнице.


Неактивность выделяется лучше, чем ходьба или бег, если рассматривать график крифой ошибок (AUC).

---

Попробуем Метод опорных векторов для сравнения

In [0]:
from sklearn import svm
clf = svm.SVC(probability=True, class_weight='balanced')
clf.fit(train_features, train_labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [0]:
predictions_svm = list(clf.predict(test_features))
pred_prob_svm = clf.predict_proba(test_features)

class_report(
    y_true=test_labels, 
    y_pred=np.asarray(predictions_svm), 
    y_score=pred_prob_svm, average='micro')

Accuracy: 0.38694638694638694
                 precision    recall  f1-score  support     pred       AUC
down the stairs   0.160682  0.143086  0.151374   2502.0   2228.0  0.609572
inactive          0.268873  0.963558  0.420429   1619.0   5802.0  0.930411
run               0.639908  0.610585  0.624903   5914.0   5643.0  0.789139
up the stairs     0.265470  0.152759  0.193927   3679.0   2117.0  0.614261
walk              0.392319  0.246646  0.302877   5591.0   3515.0  0.636381
avg / total       0.403619  0.386946  0.370988  19305.0  19305.0  0.764253


In [0]:
Точность в категории inactive, значительно ниже, чем в случайном лесе.

Из-за возможной зашумленности данных и неравномерности выборки: значений подъема/спуска по лестнице, разделение плоскостями работает хуже.